In [1]:
import feedparser
import httpx
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel
from pydantic import Field
from rich import print

In [2]:
load_dotenv()

True

In [3]:
url = "https://github.com/ava-labs/avalanchego/releases.atom"

resp = httpx.get(url)
resp.raise_for_status()


rss = feedparser.parse(resp.text)
for entry in rss["entries"]:
    for k, v in entry.items():
        print(f"{k}: {str(v)[:100]}...")
    break

id: tag:github.com,2008:Repository/246387644/v1.12.0-fuji...

guidislink: True...

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-fuji...

updated: 2024-11-19T02:31:48Z...

updated_parsed: time.struct_time(tm_year=2024, tm_mon=11, tm_mday=19, tm_hour=2, tm_min=31, tm_sec=48, tm_wday=1, 
tm...

...

authors: [{'name': 'StephenButtolph'}]...

author_detail: {'name': 'StephenButtolph'}...

author: StephenButtolph...

media_thumbnail: [{'height': '30', 'width': '30', 'url': 
'https://avatars.githubusercontent.com/u/22109487?s=60&v=4'}...

href: ...

In [4]:
max_length = 5000
model = "gpt-4o-mini"
temperature = 0
max_tokens = 1000


class Hardfork(BaseModel):
    hardfork: bool = Field(..., description="Whether this is a hardfork")
    confidence: float = Field(..., description="Confidence in the prediction, between 0 and 1")
    explanation: str = Field(..., description="Explanation why this is a hardfork or not")


def predict_hardfork(text: str) -> Hardfork:
    client = OpenAI()
    response = client.beta.chat.completions.parse(
        messages=[
            {"role": "user", "content": text},
        ],
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        response_format=Hardfork,
    )

    if not response.choices:
        return Hardfork(hardfork=False, confidence=0)

    return response.choices[0].message.parsed


for entry in rss["entries"]:
    text = str(entry)
    hf = predict_hardfork(text)

    link = entry["link"]
    print(f"link: {link}, hardfork: {hf}")

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-fuji, hardfork: hardfork=True confidence=0.9 
explanation='This release introduces significant changes through multiple Avalanche Community Proposals (ACPs) that
alter the functionality of the network, particularly in how subnets operate and fee structures. The note that this 
version cannot run on mainnet indicates a clear separation from previous versions, which is characteristic of a 
hardfork. Additionally, the requirement for all Fuji nodes to upgrade before a specific date further supports the 
notion of a hardfork, as it implies that nodes running the previous version will not be compatible with the new 
version.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13, hardfork: hardfork=False confidence=0.9 
explanation='The release notes indicate that version 1.11.13 is backwards compatible with version 1.11.0 and is 
described as optional but encouraged. This suggests that while there are new features and fixes, the core 
functionality remains intact and does not require users to upgrade to maintain compatibility. Therefore, it does 
not constitute a hard fork.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.2, hardfork: hardfork=False confidence=0.9 
explanation="The release notes indicate a merge of the 'master' branch into an update branch for version 1.11.13. 
This suggests that the changes are likely incremental updates or bug fixes rather than a significant change in 
protocol or consensus rules, which would typically characterize a hardfork."

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.1, hardfork: hardfork=False confidence=0.9 
explanation="The release notes indicate an update to 'coreth', which typically refers to improvements or bug fixes 
rather than a significant change in the protocol that would necessitate a hardfork. Hardforks usually involve 
changes that are not backward compatible, while this appears to be a routine update."

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.13-rc.0, hardfork: hardfork=False confidence=0.9 
explanation='The release notes for version v1.11.13-rc.0 indicate the addition of an optional flag to a function, 
which typically suggests a backward-compatible change rather than a hardfork. Hardforks usually involve changes 
that are not compatible with previous versions, such as altering consensus rules or transaction formats, which is 
not the case here.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.9, hardfork: hardfork=False 
confidence=0.9 explanation='The release notes for version v1.12.0-initial-poc.9 indicate the addition of metrics 
for the Etna P-chain, which suggests improvements or enhancements rather than fundamental changes to the protocol 
that would necessitate a hardfork. A hardfork typically involves changes that are not backward compatible, while 
this update appears to be an incremental improvement.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.8, hardfork: hardfork=False 
confidence=0.9 explanation='The release notes indicate a merge of branches related to logging and metrics, which 
typically suggests improvements or enhancements rather than fundamental changes to the protocol or consensus 
mechanism. A hardfork usually involves significant changes that require a split in the blockchain, while this 
appears to be a standard update or enhancement.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.7, hardfork: hardfork=True 
confidence=0.85 explanation="The release 'v1.12.0-initial-poc.7' indicates a significant update to the AvalancheGo 
project, which often includes changes that could affect the consensus mechanism or protocol rules. Given that it is
labeled as an 'initial proof of concept' (poc), it suggests that this version may introduce new features or changes
that could lead to a divergence in the blockchain, characteristic of a hardfork."

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.12.0-initial-poc.6, hardfork: hardfork=False 
confidence=0.9 explanation='The release notes indicate a merge of branches related to logging and metrics, which 
typically suggests improvements or feature additions rather than a fundamental change to the protocol or consensus 
mechanism. A hardfork usually involves significant changes that are not backward compatible, while this appears to 
be a standard update.'

link: https://github.com/ava-labs/avalanchego/releases/tag/v1.11.12, hardfork: hardfork=False confidence=0.9 
explanation='The release notes indicate that version 1.11.12 is backwards compatible with version 1.11.0 and is 
optional but strongly encouraged. This suggests that the changes made do not introduce breaking changes that would 
require users to upgrade to the new version to maintain compatibility. Therefore, it is not classified as a 
hardfork.'